# Training a model on the UD Corpus

This notebook looks at how to train a model using the Universal Dependencies Corpus. 
We will learn how to (1) download the UD Corpus, (2) train a tokenizer and a tagger model on a specific language and then (3) pack it all up in a zip model that we'll use locally.

This notebook is run on a 18.04 Ubuntu, with Python3 installed. Assume we are working in folder ``/work``. Also, let's assume that NLP-Cube is installed locally in ``/work/NLP-Cube``. If you do not have NLP-Cube installed locally (**not** using ``pip3 install nlpcube``, but direclty cloning the github repo), please first follow the [local install guide](#).

## 1. Download the UD Corpus

Let's download the Universal Dependencies Corpus. Please see [universaldependencies.org](http://www.universaldependencies.org) for more info. At the time of writing, the latest version is 2.2. For other versions please see the UD website for updated download links. Now let's download v2.2: 

In [17]:
! cd /work; curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2837/ud-treebanks-v2.2.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  252M  100  252M    0     0  6444k      0  0:00:40  0:00:40 --:--:-- 7889k


This command should download the .tgz version of the UD corpus in ``/work/ud-treebanks-v2.2.tgz``. Now, we unzip it:

In [23]:
! tar -xzf /work/ud-treebanks-v2.2.tgz -C /work

This command extracted the UD Corpus to ``/work/ud-treebanks-v2.2``. We'll use ``UD_English-ParTUT`` for training because it's a smaller dataset. A look at its contents reveals what we need to train and test our model: the train, dev and test datasets, both in raw text (.txt files) and conllu format.

In [32]:
! ls -lh /work/ud-treebanks-v2.2/UD_English-ParTUT

total 3.0M
-rwxr-xr-x 1 root root 149K Oct 16 16:44 en_partut-ud-dev.conllu
-rwxr-xr-x 1 root root  14K Oct 16 16:44 en_partut-ud-dev.txt
-rwxr-xr-x 1 root root 184K Oct 16 16:44 en_partut-ud-test.conllu
-rwxr-xr-x 1 root root  19K Oct 16 16:44 en_partut-ud-test.txt
-rwxr-xr-x 1 root root 2.3M Oct 16 16:44 en_partut-ud-train.conllu
-rwxr-xr-x 1 root root 230K Oct 16 16:44 en_partut-ud-train.txt
-rwxr-xr-x 1 root root  19K Oct 16 16:44 LICENSE.txt
-rwxr-xr-x 1 root root 4.5K Oct 16 16:44 README.md
-rwxr-xr-x 1 root root 8.2K Oct 16 16:44 stats.xml


## 2. Train a model

Next, let's train a model. We'll put everything in its own folder, say ``my_model-1.0``. We create the folder:

In [33]:
! mkdir /work/my_model-1.0

We also need an embeddings file to use in training. For this example we'll use FastText's wiki vector embeddings for English named ``wiki.en.vec``, downloaded from [here](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) that we've put in ``/work/wiki.en.vec``. 

We're ready to start training. Because training might take a long time, please open up a ``screen`` or put the train process in background in case your console disconnects for any reason.

We'll train first a tokenizer with default parameters and then a tagger with custom parameters.

### 2.1. Training a default tokenizer

Let's train our model! We simply cd to ``NLP-Cube`` and run a one-liner in a shell: 

```
python3 /work/NLP-Cube/cube/main.py --train=tokenizer --train-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-train.conllu --dev-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-dev.conllu --raw-train-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-train.txt --raw-dev-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-dev.txt --embeddings /work/wiki.en.vec --store /work/my_model-1.0/tokenizer --batch-size 1000 --set-mem 8000 --autobatch --patience 1

```


Let's look at the switches in detail ( you can see the full help with the ``--help`` switch): 

```
  --train=TRAIN             select which model to train: tagger, parser,
                            lemmatizer, tokenizer, mt, compound, ner
```

Because we want to train a tokenizer model, we'll pass ``--train=tokenizer``.
```
  --train-file=TRAIN_FILE   location of the train dataset
  --dev-file=DEV_FILE       location of the dev dataset  
```
Here we pass the path to the train and dev **.conllu** files. In this case, the train file is ``/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-train.conllu``

Because we train a tokenizer that has to learn to transform raw text (seen as a string) into tokens (given in the .conllu format), we need to specify the --raw-train-file and --raw-dev-file as well. Note that these switches are only relevant for the tokenizer; all other tasks (lemmatizer, tagger, etc.) need only the .conllu files.
```
  --raw-train-file=RAW_TRAIN_FILE   location of the raw train file
  --raw-dev-file=RAW_DEV_FILE       location of the raw dev file
```
Next, we tell NLP-Cube where to find the embeddings file:
```
  --embeddings=EMBEDDINGS           location of the pre-computed word embeddings file
```
We then tell NLP-Cuve where to store the trained model:
```
  --store=OUTPUT_BASE   output base for model location
```
Please note that ``--store`` is not a folder path, but rather a **prefix**. For example, ``--store /work/my_model-1.0/tokenizer`` will create in the ``my_model-1.0`` folder several files that begin with _tokenizer_, such as _tokenizer.encodings_, _tokenizer-tok.bestAcc_, etc.

Other switches include ``--patience`` which specifies the number of epochs to stop the training after there is no improvement on the dev set (early stopping condition). In this example we'll set the patience to 1 epoch; for normal training we recommend setting patience anywhere from 20 to 100 epochs. We recommend using a **larger ``--batch-size`` of 1000** and ``--autobatch``ing to speed up training. For autobatching we need to reserve memory in advance with ``--set-mem`` (given as an int: 4000 = 4000 MB). If there is a GPU available, place the training on it with the ``--use-gpu`` flag. We'll see the ``--config`` flag when training a tagger with custom configuration. For now, not specifying a config will create a model with default network parameters.
```
  --patience=ITTERS     no improvement stopping condition
  --config=CONFIG       configuration file to load  
  --batch-size=BATCH_SIZE
  --set-mem=MEMORY      preallocate memory for batch training (default 2048)
  --autobatch           turn on/off dynet autobatching
  --use-gpu             turn on/off GPU support
```

Let's training the model and redirect all stdout messages in a log file with ``&> /work/my_model-1.0/tokenizer.log``. **Please ensure your current working dir is ``/work/NLP-Cube``, otherwise some relative imports won't load correctly**.

```
python3 /work/NLP-Cube/cube/main.py --train=tokenizer --train-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-train.conllu --dev-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-dev.conllu --raw-train-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-train.txt --raw-dev-file=/work/ud-treebanks-v2.2/UD_English-ParTUT/en_partut-ud-dev.txt --embeddings /work/wiki.en.vec --store /work/my_model-1.0/tokenizer --batch-size 1000 --set-mem 8000 --autobatch --patience 1 &> /work/my_model-1.0/tokenizer.log
```

Training might take a long time. Check the log to see how training progresses. If successful, the last lines of the log file will look like: 
```
Starting epoch 5
	shuffling training data... done
	training... 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 avg_loss=7.148099940487884 execution_time=461.953825712204
	evaluating on devset... token_accuracy=99.65092779717068 , sentence_accuracy=100.0
	Storing /work/my_model-1.0/tokenizer-ss.last
	Storing /work/my_model-1.0/tokenizer-tok.last
Training is done with devset sentence tok = 99.74264705882354 and sentence = 100.0 
```
This means that we have had a successful training run with a sentence tokenization accuracy of 99.74% and sentence segmentation accuracy of 100%.

A few files have been created in 'my_model-1.0'; the important ones are _tokenizer.log_ with the training progress, _tokenizer-tok.bestAcc_ which is the tokenization model, _tokenizer-ss.bestAcc_ which is the sentence segmentation model, _tokenizer.conf_, and the _tokenizer.encodings_ which contains word lists relevant to the models. We'll see how to use these files later on.


## 2.2. Training a custom tagger

Now let's traing a tagger using custom config. In ``NLP-Cube/examples`` there are a number of default conf files. The _tagger.conf_ looks like:

```
[TaggerConfig]
aux_softmax_layer = 2
input_dropout_prob = 0.33
input_size = 100
layer_dropouts = [0.5, 0.5, 0.5]
layers = [200, 200]
presoftmax_mlp_dropouts = [0.5]
presoftmax_mlp_layers = [500]

```

In this tutorial we won't go into details about the structure of the tagger, but let's say that we want to change the ``layers`` parameter, and instead of 2 BiLSTMs of size 200 we want to use 3 BiLSTMs of size 100. Copy the __tagger.conf__ file from the ``examples/`` folder to ``my_model-v1.0`` and change the layers line to ``layers = [100, 100, 100]``. Feel free to experiment with other values. 

Now, let's run the tagger:

```
python3 /work/NLP-Cube/cube/main.py --train=tagger --train-file=/work/ud-treebanks-v2.2/UD_English-EWT/en_ewt-ud-train.conllu --dev-file=/work/corpus/ud-treebanks-v2.2/UD_English-EWT/en_ewt-ud-dev.conllu --embeddings /work/wiki.en.vec --store /work/my_model-1.0/tagger --patience 1 --config /work/my_model-1.0/tagger.conf --batch-size 1000 &> /work/my_model-1.0/tagger.log
```

Note the ``-config /work/my_model-1.0/tagger.conf`` parameter. For our tokenizer we didn't pass this parameter so NLP-Cube has created a default _tokenizer.conf_ file for us. Because now we specify the conf file, the tagger will read it and adjust its internal structure accordingly. Also, for this example we only want to specify the ``--batch-size`` and let NLP-Cube manage memory automatically. Training should finish with the log file ending with:

```
Training is done with devset accuracy=( UPOS=0.9549900596421471 , XPOS=0.950934393638171 , ATTRS=0.9617097415506958 )
```

Just like with the tokenizer, we have a number of _tagger*_ important files. Because we'll use a script that will automatically package them into a single model, we don't have to worry about them. Also, we can use them immediately, as shown in the next tutorial.

### Note about file-naming:

Please note that the ``--store /work/my_model-1.0/tagger`` parameter need to end with : ``/tokenizer`` when training a tokenizer, ``/compound`` when training a compound word expander, ``/lemmatizer`` for lemmatization and ``/parsing`` for training a parser. 

We need this convention ...



---

The next tutorial shows how to [use a locally trained model](#).